In [1]:
# Librerías de utilidad para manipulación y visualización de datos.
import pandas as pd
from pydantic import BaseModel
import numpy as np
import fastapi
import json
import pickle
from keras.models import load_model
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
import pydantic
import keras
import nltk
import tensorflow 
print(pd.__version__)
print(pydantic.__version__)
print(np.__version__)
print(fastapi.__version__)
print(json.__version__)
#print(pickle.__version__)
print(keras.__version__)
print(nltk.__version__)
print(tensorflow.__version__)

2.2.2
2.6.1
1.23.5
0.110.1
2.0.9
3.2.1
3.8.1
2.16.1


In [8]:
%%writefile requirements.txt
fastapi==0.82.0
uvicorn==0.19.0
pandas == 2.2.2
pydantic == 2.6.1
numpy==1.23.5
json==2.0.9
keras==3.2.1
nltk==3.8.1
tensorflow==2.16.1

Writing requirements.txt


In [9]:
%%writefile railway.json
{
  "$schema": "https://railway.app/railway.schema.json",
  "build": {
    "builder": "NIXPACKS"
  },
  "deploy": {
    "startCommand": "uvicorn main:app --host 0.0.0.0 --port $PORT",
    "restartPolicyType": "ON_FAILURE",
    "restartPolicyMaxRetries": 10
  }
}

Writing railway.json


In [ ]:
%%writefile main.py
import pandas as pd
from pydantic import BaseModel
import numpy as np
from fastapi import FastAPI
import json
import pickle
from keras.models import load_model
from nltk.tokenize import word_tokenize
from tensorflow.keras.preprocessing.sequence import pad_sequences

def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text])
    tokens = word_tokenize(text)
    return ' '.join(tokens)

def vectorize(X,tokenizer):
    # Vectorización de X
    X_vect = tokenizer.texts_to_sequences(X)
    # Longitud vectorizada del primer elemento
    len(X_vect)
    # Dejamos los vectores de igual longitud
    # Adicionamos padding en caso de ser necesario
    max_length = 100
    X_vect = pad_sequences(X_vect, maxlen=max_length, padding='post')
    return X_vect


# CLASE CALIFICADA ApiInput
class ApiInput(BaseModel):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    features: str
    ### FIN DEL CÓDIGO ###

# CLASE CALIFICADA ApiOutput
class ApiOutput(BaseModel):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    forecast: str
    ### FIN DEL CÓDIGO ###

app = FastAPI()

# Reemplace esto con su implementación:
@app.post("/predict")
async def predict(data: ApiInput) -> ApiOutput:
    
    model = load_model('model_conv1d.h5') # cargamos el modelo.
    
    with open('labels.json', 'r') as openfile:
        # Reading from json file
        json_object = json.load(openfile)

    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    input_series = pd.Series(data.features)
    input_preprocess = input_series.apply(preprocess)
    input_vect = vectorize(input_preprocess,tokenizer)
    predictions = np.argmax(model.predict(input_vect),axis=1)
    prediction = ApiOutput(forecast=list(json_object.keys())[predictions[0]])
    return prediction

In [2]:
# 1. Estructura Test 1 Funciona
# Definimos una función para el preprocesamiento del texto convertido a trigramas (codones)
def preprocess(text):
    text = text.lower()
    text = ''.join([word for word in text])
    tokens = word_tokenize(text)
    return ' '.join(tokens)

def vectorize(X,tokenizer):
    # Vectorización de X
    X_vect = tokenizer.texts_to_sequences(X)
    # Longitud vectorizada del primer elemento
    len(X_vect)
    # Dejamos los vectores de igual longitud
    # Adicionamos padding en caso de ser necesario
    max_length = 100
    X_vect = pad_sequences(X_vect, maxlen=max_length, padding='post')
    return X_vect


# CLASE CALIFICADA ApiInput
class ApiInput(BaseModel):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    features: str
    ### FIN DEL CÓDIGO ###

# CLASE CALIFICADA ApiOutput
class ApiOutput(BaseModel):
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    forecast: str
    ### FIN DEL CÓDIGO ###

def get_api_input(features: str) -> ApiInput:
    return ApiInput(features=features)

def get_api_output(forecast: str) -> ApiOutput:
    return ApiOutput(forecast=forecast)

def predict(data: ApiInput) -> ApiOutput:
    ### ESCRIBA SU CÓDIGO AQUÍ ###
    model = load_model('model_conv1d.h5') # cargamos el modelo.
    
    with open('labels.json', 'r') as openfile:
        # Reading from json file
        json_object = json.load(openfile)

    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
    
    input_series = pd.Series(data.features)
    input_preprocess = input_series.apply(preprocess)
    input_vect = vectorize(input_preprocess,tokenizer)
    predictions = np.argmax(model.predict(input_vect),axis=1)
    prediction = ApiOutput(forecast=list(json_object.keys())[predictions[0]])
    return prediction

#TEST_CELL
test1 = "ttc tcg cgt gta tag agt gtg tgt gtg tgg ggg ggg ggc gct ctg tgt gtg tgc gcg cgg gga gat att tta tat ata tag aga gag agg ggt gtt ttg tgg ggt gtg tgc gcc ccg cgt gta tac aca cag agc gcc cct ctg tgc gct ctt ttt ttg tgc gcc cct cta tat atg tga gaa aat ata tac act ctc tcg cgt gtt ttc tcg cgg ggc gcg cgc gca caa aaa aag agg ggg ggg ggt gtg tgt gtg tgc gca cag aga gac acg cgg gga gat ata tac aca cat att ttt ttg tgc gct ctg tgc gcg cgt gtc tcc ccc cct cta taa aac aca cac acg cgt gtt ttc tcc ccg cgc gcc cct cta taa aat atg"
inp = ApiInput(features=(test1.upper()))
#inp = ApiInput(features='GGG GGT GTT TTT TTT TTG TGG GGA GAC ACT CTA TAA AAA AAA AAT ATC TCA CAA AAG AGG GGA GAG AGG GGT GTG TGG GGG GGT GTA TAA AAG AGG GGA GAG AGG GGC GCA CAG AGC GCG CGA GAA AAA AAG AGC GCG CGG GGC GCT CTG TGC GCG CGA GAA AAG AGG GGC GCA CAG AGC GCC CCG CGG GGC GCA CAA AAA AAG AGC GCC CCG CGC GCA CAT ATT TTA TAG AGG GGC GCG CGC GCC CCG CGT GTT TTT TTG TGC GCG CGA GAG AGG GGC GCA CAG AGT GTA')
print(type(inp), inp.features)

pred = predict(inp)
display(pred)

c:\Python310\lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
c:\Python310\lib\site-packages\keras\src\layers\core\embedding.py:89: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


<class '__main__.ApiInput'> TTC TCG CGT GTA TAG AGT GTG TGT GTG TGG GGG GGG GGC GCT CTG TGT GTG TGC GCG CGG GGA GAT ATT TTA TAT ATA TAG AGA GAG AGG GGT GTT TTG TGG GGT GTG TGC GCC CCG CGT GTA TAC ACA CAG AGC GCC CCT CTG TGC GCT CTT TTT TTG TGC GCC CCT CTA TAT ATG TGA GAA AAT ATA TAC ACT CTC TCG CGT GTT TTC TCG CGG GGC GCG CGC GCA CAA AAA AAG AGG GGG GGG GGT GTG TGT GTG TGC GCA CAG AGA GAC ACG CGG GGA GAT ATA TAC ACA CAT ATT TTT TTG TGC GCT CTG TGC GCG CGT GTC TCC CCC CCT CTA TAA AAC ACA CAC ACG CGT GTT TTC TCC CCG CGC GCC CCT CTA TAA AAT ATG
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step


ApiOutput(forecast='Anti_Viral')